# Implenting an Array API to use with Scikit-learn


In this tutorial, we will create an object that implements the Array API and use it in the `LinearDiscriminantAnalysis` example that is in the [scikit-learn docs](https://scikit-learn.org/stable/modules/array_api.html).

First, let's try LDA with normal numpy arrays.

We take a set of input vector and reduce the dimensionality to 1.

In [1]:
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import config_context

iris = datasets.load_iris()

X = iris.data
y = iris.target

def fit(X, y):
    with config_context(array_api_dispatch=True):
        lda = LinearDiscriminantAnalysis(n_components=2)
        X_r2 = lda.fit(X, y).transform(X)
        return X_r2

    target_names = iris.target_names
    plt.figure()
    colors = ["navy", "turquoise", "darkorange"]

    plt.figure()
    for color, i, target_name in zip(colors, [0, 1, 2], target_names):
        plt.scatter(
            X_r2[y == i, 0], X_r2[y == i, 1], alpha=0.8, color=color, label=target_name
        )
    plt.legend(loc="best", shadow=False, scatterpoints=1)
    plt.title("LDA of IRIS dataset")

    plt.show()
# fit(X, y)

In [2]:
import torch

# fit(torch.asarray(X), torch.asarray(y))

Now let's try to make a NDArray object that implements the Array API.

In [5]:
%load_ext autoreload
%autoreload complete

In [7]:
from egglog.exp.array_api import *


X_arr = NDArray.var("X")
y_arr = NDArray.var("y")

# Add values for the constants
egraph.register(
    rewrite(X_arr.dtype, runtime_ruleset).to(convert(X.dtype, DType)),
    rewrite(y_arr.dtype, runtime_ruleset).to(convert(y.dtype, DType)),
    rewrite(isfinite(sum(X_arr)).to_bool(), runtime_ruleset).to(TRUE),
    rewrite(isfinite(sum(y_arr)).to_bool(), runtime_ruleset).to(TRUE),
    rewrite(X_arr.shape, runtime_ruleset).to(convert(X.shape, TupleInt)),
    rewrite(y_arr.shape, runtime_ruleset).to(convert(y.shape, TupleInt)),
    rewrite(X_arr.size, runtime_ruleset).to(Int(X.size)),
    rewrite(y_arr.size, runtime_ruleset).to(Int(y.size)),
    rewrite(unique_values(y_arr).shape).to(TupleInt(Int(3))),
)


res = fit(X_arr, y_arr)

# X_obj, y_obj = egraph.save_object(X), egraph.save_object(y)

# X_arr = NDArray(X_obj)
# y_arr = NDArray(y_obj)
# TODO: Make index type be a list. Each item in the list can be a slice, an int,

DType.float64 == NDArray.var("X").dtype
  -> DType.float64 == NDArray.var("X").dtype
     -> TRUE
asarray(NDArray.var("X")).ndim == Int(0)
  -> NDArray.var("X").ndim == Int(0)
     -> FALSE
asarray(NDArray.var("X")).ndim == Int(1)
  -> NDArray.var("X").ndim == Int(1)
     -> FALSE
asarray(NDArray.var("X")).ndim >= Int(3)
  -> NDArray.var("X").ndim >= Int(3)
     -> FALSE
asarray(asarray(NDArray.var("X"))).dtype == DType.object
  -> NDArray.var("X").dtype == DType.object
     -> FALSE
isdtype(asarray(asarray(NDArray.var("X"))).dtype, (IsDtypeKind.string("real floating") | (IsDtypeKind.string("complex floating") | IsDtypeKind.NULL)))
  -> isdtype(NDArray.var("X").dtype, (IsDtypeKind.string("real floating") | IsDtypeKind.string("complex floating")))
     -> TRUE
isfinite(sum(asarray(asarray(NDArray.var("X"))))).to_bool()
  -> isfinite(sum(NDArray.var("X"))).to_bool()
     -> TRUE
asarray(NDArray.var("X")).shape.length()
  -> NDArray.var("X").ndim
     -> Int(2)
asarray(NDArray.var("X")).s

TypeError: Cannot convert slice(None, None, None) (<class 'slice'>) to TypeRefWithVars(name='Int', args=())